In [22]:
import sys
import os
sys.path.append("..")
from knool.remote_sensing.satellite import tles_for_asat
from knool.geodata_processor import geo_info
import pandas as pd
from skyfield.api import load
from datetime import datetime as dt
from datetime import timedelta
from skyfield.api import utc
import numpy as np
from osgeo import gdal, osr, ogr

In [23]:
##READ tle file. https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

tles = load.tle_file("../knool/remote_sensing/data/two_line_elements/sat40697.txt") # TLE読み込み
satellite=tles[0]
sgp4_model = satellite.model

print("衛星名: ", satellite.name)
print("元期: ", satellite.epoch.utc_jpl())
print("衛星番号: ", sgp4_model.satnum)
print("エポック年: ", sgp4_model.epochyr)
print("エポック日: ", sgp4_model.epochdays)
print("エポックのユリウス日: ", sgp4_model.jdsatepoch)
print("ndot: ", sgp4_model.ndot)
print("nddot: ", sgp4_model.nddot)
print("弾道抗力係数 B*: ", sgp4_model.bstar)
print("軌道傾斜角[rad]: ", sgp4_model.inclo)
print("昇交点赤経[rad]: ", sgp4_model.nodeo)
print("離心率: ", sgp4_model.ecco)
print("近地点引数[rad]: ", sgp4_model.argpo)
print("平均近点角[rad]: ", sgp4_model.mo)
print("平均運動[rad/min]: ", sgp4_model.no_kozai)

衛星名:  None
元期:  A.D. 2015-Jun-23 03:50:23.3849 UTC
衛星番号:  40697
エポック年:  15
エポック日:  174.15999288
エポックのユリウス日:  2457196.5
ndot:  -1.333237623051224e-12
nddot:  0.0
弾道抗力係数 B*:  0.0
軌道傾斜角[rad]:  1.7206276070908537
昇交点赤経[rad]:  4.337586504835915
離心率:  7.98e-05
近地点引数[rad]:  4.953066393576204
平均近点角[rad]:  2.5043519997356394
平均運動[rad/min]:  0.06244205289471824


In [32]:
#get lat,lon,elevation from a tle
ts = load.timescale()  # skyfiled上で時刻を扱うモジュール
tle=tles[0]
date1 = dt(2018, 2, 1, 12, 15, 30, 2000, tzinfo=utc)
date1_sf=ts.from_datetime(date1)

geocentric=tle.at(date1_sf) #get xyz
subpoint = geocentric.subpoint()
lat = subpoint.latitude.degrees
lon = subpoint.longitude.degrees
ele = subpoint.elevation.m
print(geocentric.velocity)
print(geocentric.position)
print(geocentric.xyz)
print(lat,lon,ele)

[-0.00141461  0.00399833  0.00070632] au/day
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
-77.27650275778583 -162.68188667619526 815689.5183005631


In [38]:
#convert lla/ecef from ecef/lla using our package.
print(lat,lon,ele)
x,y,z=geo_info.convert_lla_to_ecef(lat,lon,ele)
print(x,y,z) # meter
lat,lon,ele=geo_info.convert_ecef_to_lla(x,y,z)
print(lat,lon,ele)

-77.27650276073706 -162.68188667619526 815689.5199458357
-1516878.350533145 -472981.1087535418 -6995304.200849125
-77.27650276368828 -162.68188667619526 815689.5215911083


In [39]:
#create a kml file of a orbit line
infile=r"../knool/remote_sensing/data/two_line_elements/sat40697.txt"
test=tles_for_asat(infile)
test.calc_positions_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:26: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(date), method='nearest')]]


In [40]:
#Creates a kml file of a line that are a specified distance away from the orbital line.
test.calc_buff_positions_between("2011-07-01 10:00:00","2011-07-01 11:40:00",2,1000000,ori="left")
test.export("../../test_data/output/test2.kml")

In [44]:
#Creates a kml file of a polygon
test.calc_buff_area_between("2011-07-01 10:00:00","2011-07-01 11:00:00",0.5,1000000,ori="left")
test.export("../../test_data/output/test3.kml")

[[55.282280597897035, 17.01449688038268],
 [53.55454800512188, 16.10530762463548],
 [51.82152660049911, 15.257519376783344],
 [50.08382982920377, 14.463141034047196],
 [48.34197199091391, 13.715456131258575],
 [46.59638812458671, 13.008779975271125],
 [44.84744949579037, 12.338269291818458],
 [43.095475714493084, 11.699771777407811],
 [41.34074433837252, 11.089706266024729],
 [39.58349852535947, 10.50496655457701],
 [37.823953171664385, 9.942843660058948],
 [36.062299884564844, 9.400962548317743],
 [34.298711015253076, 8.877230296155743],
 [32.53334296334, 8.369793356213508],
 [30.76633889219415, 7.87700210945273],
 [28.997830962237913, 7.3973812867581215],
 [27.227942185935788, 6.929605147495366],
 [25.45678797368684, 6.472476530755647],
 [23.684477400510225, 6.024909069950655],
 [21.911114293969135, 5.58591201576764],
 [20.136798107873695, 5.154577197634774],
 [18.36162468325895, 4.730067765886927],
 [16.58568686551335, 4.311608401315783],
 [14.809075024145821, 3.8984767471702733],
 